In [10]:
from typing import Dict,List
from typing import Literal
from collections import defaultdict
from functools import reduce
import pandas as pd

In [11]:
data_type=Literal['all','attitude','altitude','speed']


def log_data(fichero:str=None,mode:data_type=None):
    simple_read:list[data_type]=['attitude','altitude','speed']

    actual_name={
        'attitude':'mavlink_attitude_t',
        'altitude':'mavlink_ahrs2_t',
        'speed':'mavlink_vfr_hud_t'
    }

    if not data_type:
        return 

    
    if mode in simple_read:
        examinees=[mode]
    elif mode =='all':
        examinees=simple_read

    databases=[]

    with open(fichero,'r') as file:

        def dataf(examinee):
            Registro=defaultdict(list)
            
            for line in file:
                if actual_name[examinee] in line:
                    parts=line.strip().split()

                    Fecha=parts[0]
                    Hora=parts[1]

                    lectura=parts[11:-5]
                
                    Registro['Fecha'].append(Fecha)
                    Registro['Hora'].append(Hora)
                        
                    for j in range(0,len(lectura),2):
                        Registro[str(lectura[j])].append(float(lectura[j+1].replace(',','.')))
            file.seek(0)
            return Registro


        for examinee in examinees:

            tolon=dataf(examinee)    
            
            cosa=pd.DataFrame(tolon)
            if cosa.empty:
                print('nasa')
            if not cosa.empty:
                databases.append(cosa)

            if len(examinees)==1:
                return databases[0]
            
    resultado = reduce(lambda left, right: pd.merge(left, right, on=['Fecha', 'Hora'], how='outer'), databases)

    return resultado


In [12]:
AS=log_data("2024-10-07 17-20-41.txt",'all')

In [13]:
def log_attitude(fichero:str=None,Registro:Dict[str,List[float]]={}) -> Dict[str,List[float]]:
    """
    Le os contidos do ficheiro referentes a actitude no .txt extraido de ardupilot

    :param Fichero:
    Enderezo do fichero a leer
    
    :param Registro:
    Diccionario contentendo Fecha,hora, actitude e velocidades correspondentes medidas

    :return: Rexistro das actitudes e velocidades angulares


    """

    with open(fichero,'r') as file:
        for line in file:
            if 'mavlink_attitude_t' in line:
                Fecha=line.strip().split()[0]
                Hora=line.strip().split()[1]

                lectura=line.strip().split()[11:-5]
                
                if not Registro:
                    Registro['Fecha']=[]
                    Registro['Hora']=[]
                    for i in range(0,len(lectura),2):
                        Registro[lectura[i]]=[]
                Registro['Fecha'].append(Fecha)
                Registro['Hora'].append(Hora)
                for j in range(0,len(lectura),2):
                    Registro[str(lectura[j])].append(float(lectura[j+1].replace(',','.')))
    return Registro

In [14]:
def log_altitude(fichero:str=None,Registro:Dict[str,List[float]]={}) -> Dict[str,List[float]]:
    """
    Le os contidos do ficheiro referentes a altitude no .txt extraido de ardupilot

    :param Fichero:
    Enderezo do fichero a leer
    
    :param Registro:
    Diccionario contentendo Fecha,hora, altitude e medidas asociadas

    :return: Rexistro das altitudes e diversas vairables


    """

    with open(fichero,'r') as file:
        for line in file:
            if 'mavlink_ahrs2_t' in line:
                Fecha=line.strip().split()[0]
                Hora=line.strip().split()[1]

                lectura=line.strip().split()[11:-5]
                
                if not Registro:
                    Registro['Fecha']=[]
                    Registro['Hora']=[]
                    for i in range(0,len(lectura),2):
                        Registro[lectura[i]]=[]
                Registro['Fecha'].append(Fecha)
                Registro['Hora'].append(Hora)
                for j in range(0,len(lectura),2):
                    Registro[str(lectura[j])].append(float(lectura[j+1].replace(',','.')))

    return Registro

In [15]:
def log_speed(fichero:str=None,Registro:Dict[str,List[float]]={}) -> Dict[str,List[float]]:
    """
    Le os contidos do ficheiro referentes a velocidad na hud no .txt extraido de ardupilot

    :param Fichero:
    Enderezo do fichero a leer
    
    :param Registro:
    Diccionario contentendo Fecha,hora, medicions de velocidade

    :return: Rexistro de velocidades


    """

    with open(fichero,'r') as file:
        for line in file:
            if 'mavlink_vfr_hud_t' in line:
                Fecha=line.strip().split()[0]
                Hora=line.strip().split()[1]

                lectura=line.strip().split()[11:-5]
                
                if not Registro:
                    Registro['Fecha']=[]
                    Registro['Hora']=[]
                    for i in range(0,len(lectura),2):
                        Registro[lectura[i]]=[]
                Registro['Fecha'].append(Fecha)
                Registro['Hora'].append(Hora)
                for j in range(0,len(lectura),2):
                    Registro[str(lectura[j])].append(float(lectura[j+1].replace(',','.')))

    return Registro

In [16]:
if __name__=='__main__':
    Attitudes=log_attitude("2024-10-07 17-20-41.txt")
    Altitudes=log_altitude("2024-10-07 17-20-41.txt")
    Speeds=log_speed("2024-10-07 17-20-41.txt")

In [18]:
Attitudes_2=log_data("2024-10-07 17-20-41.txt",mode='attitude')